<a href="https://colab.research.google.com/github/Billy-Drunkenstein/CSI-300-RL-Agent/blob/Bill/Bill/Neural%20Network%20Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
import random

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

In [3]:
data = pd.read_csv('000300.SH.csv', index_col = 0, parse_dates = True)
data.head()

,OPEN,HIGH,LOW,CLOSE,VWAP,VOLUME,AMT,TURN,TOTAL_SHARES,FREE_FLOAT_SHARES,MKT_CAP_ARD,MKT_FREESHARES,PE_TTM,VAL_PB_WGT,DIVIDENDYIELD2
2005-04-08,984.665,1003.699,979.529,1003.445,6.199041,1.476253e+09,9.151350e+09,1.3953,4.390040e+11,1.047090e+11,2.580696e+12,6.274793e+11,15.6050,2.0622,1.8341
2005-04-11,1003.879,1008.735,992.773,995.421,6.548813,1.593607e+09,1.043623e+10,1.5062,4.390040e+11,1.047090e+11,2.558621e+12,6.222988e+11,15.4991,2.0377,1.8482
2005-04-12,993.711,993.711,978.204,978.697,6.336242,1.022619e+09,6.479563e+09,0.9665,4.392522e+11,1.047090e+11,2.518147e+12,6.116571e+11,15.1283,2.0008,1.9026
2005-04-13,987.950,1006.499,987.950,1000.900,6.240541,1.607169e+09,1.002960e+10,1.5190,4.392522e+11,1.047090e+11,2.572078e+12,6.255622e+11,15.5734,2.0453,1.8531
2005-04-14,1004.640,1006.416,985.578,986.975,6.035532,1.294571e+09,7.813425e+09,1.2224,4.395948e+11,1.048114e+11,2.548100e+12,6.169851e+11,15.4431,2.0256,1.8396


In [4]:
WINDOWS = {"3y": 756, "1y": 252, "6m": 126, "3m": 63, "1m": 21, "2w": 10}

new_columns = {}
for col in data.columns:
    for window_name, window in WINDOWS.items():
        new_columns[f"{col}_{window_name}_MEAN"] = data[col].rolling(window=window).mean()
        new_columns[f"{col}_{window_name}_STD"] = data[col].rolling(window=window).std()
        new_columns[f"{col}_{window_name}_SKEW"] = data[col].rolling(window=window).skew()

    for lag in range(1, 3):
        new_columns[f"{col}_LAG_{lag}"] = data[col].shift(lag)

data = pd.concat([data, pd.DataFrame(new_columns)], axis = 1)
data.dropna(inplace = True)

In [5]:
data.head()

,OPEN,HIGH,LOW,CLOSE,VWAP,VOLUME,AMT,TURN,TOTAL_SHARES,FREE_FLOAT_SHARES,...,DIVIDENDYIELD2_3m_STD,DIVIDENDYIELD2_3m_SKEW,DIVIDENDYIELD2_1m_MEAN,DIVIDENDYIELD2_1m_STD,DIVIDENDYIELD2_1m_SKEW,DIVIDENDYIELD2_2w_MEAN,DIVIDENDYIELD2_2w_STD,DIVIDENDYIELD2_2w_SKEW,DIVIDENDYIELD2_LAG_1,DIVIDENDYIELD2_LAG_2
2008-05-20,3911.096,3946.441,3698.554,3710.818,16.263919,3.970263e+09,6.457203e+10,1.2424,1.605192e+12,2.829967e+11,...,0.067731,-0.317431,0.619567,0.033052,0.080456,0.62938,0.029158,0.258369,0.6445,0.6466
2008-05-21,3668.181,3801.067,3591.566,3783.049,15.868166,4.359616e+09,6.917911e+10,1.3618,1.606971e+12,2.835677e+11,...,0.067008,-0.307293,0.620014,0.033683,0.114955,0.63629,0.029658,-0.174770,0.6822,0.6445
2008-05-22,3732.641,3783.849,3704.157,3711.444,16.268236,4.074272e+09,6.628121e+10,1.2715,1.611218e+12,2.838894e+11,...,0.066529,-0.298757,0.620771,0.034783,0.176265,0.64474,0.027330,-0.383638,0.6693,0.6822
2008-05-23,3697.890,3741.060,3629.009,3675.147,15.708758,3.344043e+09,5.253076e+10,1.0413,1.612052e+12,2.849255e+11,...,0.066290,-0.273744,0.622424,0.037130,0.283645,0.65300,0.026453,-0.574637,0.6757,0.6693
2008-05-26,3643.012,3643.012,3555.762,3559.217,16.154744,2.818019e+09,4.552437e+10,0.8768,1.612599e+12,2.853157e+11,...,0.066673,-0.247759,0.625871,0.040813,0.312168,0.66300,0.022846,0.293368,0.6876,0.6757


# Trade Parameters

In [6]:
transaction_cost = 0.0015

# Reinforcement Learning Parameters

In [7]:
gamma = 0.9     # Reward Decay Coefficient

# CSI 300 Trading Environment

In [8]:
class CSI300TradingEnv(Env):

    def __init__(self, data: pd.DataFrame, window_size: int = 1, lookback: int = 50, ignored: int = 3, mode: str = "train",
                 transaction_cost: float = 0.0015, gamma: float = 0.9):
        super().__init__()

        self.data = data
        self.raw_data = data.copy(deep = True)
        self.window_size = window_size
        self.num_features = self.data.shape[1]
        self.reward_length = lookback
        self.ignored = ignored

        # Simplified Action Space: 0 = Hold
                                #  1 = Purchase
                                #  2 = Liquidate
        self.action_space = Discrete(3)

        self.transaction_cost = transaction_cost

        self.gamma = gamma

        # Action Mapping for Reward Calculation
        self.action_mapping = {0: -1, 1: 0, 2: 1}

        self.action_label = {
            0: "hold",
            1: "enter",
            2: "liquidate"
        }

        self.observation_space = Box(
            low=-np.inf,
            high=np.inf,
            shape=(self.window_size * self.num_features,),
            dtype=np.float32
        )

        self.index_position = self.window_size
        self.position = -1
        self.done = False

        self.position_history = [] # len = len(return_history) + 1
        self.return_history = []

        self.cumulative_log_return = 0.0

        self.data_all = data
        self.splits = {
            'train': (0, int(0.75 * len(self.data_all))),
            'test': (int(0.75 * len(self.data_all)), int(0.9 * len(self.data_all))),
            'oos': (int(0.9 * len(self.data_all)), len(self.data_all))
        }

        train_start, train_end = self.splits['train']
        scaler = StandardScaler()
        scaler.fit(self.data_all.iloc[train_start:train_end])

        scaled_values = scaler.transform(self.data_all)
        self.data_all = pd.DataFrame(scaled_values, columns=self.data_all.columns, index=self.data_all.index)

        self.mode = mode
        self._set_data_for_mode()


        # Temporary Debug Variables
        self.return_history_dated = []     # List of (date, return)
        self.position_history_dated = []   # List of (date, position)

    def _set_data_for_mode(self):
        start, end = self.splits[self.mode]
        self.data = self.data_all.iloc[start:end]

    def set_mode(self, mode: str):
        if mode not in self.splits:
            raise ValueError(f"Invalid mode: {mode}. Must be one of {list(self.splits.keys())}")
        self.mode = mode
        self._set_data_for_mode()
        self.reset()

    def change_lookback(self, new_lookback: int):
        self.reward_length = new_lookback

    def reset(self):
        self.index_position = self.window_size
        self.position = -1
        self.done = False
        self.position_history = []
        self.return_history = []
        self.cumulative_log_return = 0.0

        self.return_history_dated = []
        self.position_history_dated = []

        return self._get_observation()

    def step(self, action):
        mapped_action = self.action_mapping[action]

        # Update Portfolio Status:
        if action == 1:  # enter
            self.position = 1
        elif action == 2:  # liquidate
            self.position = -1

        date_today = self.data.index[self.index_position]
        date_prev = self.data.index[self.index_position - 1]

        open_today = self.raw_data.loc[date_today, "OPEN"]
        open_prev = self.raw_data.loc[date_prev, "OPEN"]
        daily_return = open_today / open_prev - 1

        self.position_history.append(self.position)
        self.position_history_dated.append((date_today, self.position))

        # Skip the first day
        if self.index_position > self.window_size:
            self.return_history.append(daily_return)
            self.return_history_dated.append((date_prev, daily_return))

        if len(self.return_history) > self.reward_length:
            self.return_history.pop(0)
            self.return_history_dated.pop(0)
        if len(self.position_history) > self.reward_length + 1:
            self.position_history.pop(0)
            self.position_history_dated.pop(0)

        log_daily_return = np.log(1 + daily_return) if self.position == 1 else 0.0

        if len(self.position_history) >= 2:
            prev_position = self.position_history[-2]
            transaction_cost = self.transaction_cost if prev_position != self.position else 0.0
        else:
            transaction_cost = 0.0

        self.cumulative_log_return += log_daily_return - transaction_cost

        # Reward and Return
        reward = 0.0
        total_return = 0.0
        cutoff = self.reward_length - self.ignored
        if len(self.return_history_dated) >= self.reward_length and len(self.position_history_dated) >= self.reward_length + 1:
            returns_slice = self.return_history_dated[:cutoff]
            positions_slice = self.position_history_dated[:cutoff]

            dates_match = all(r[0] == p[0] for r, p in zip(returns_slice, positions_slice))
            if dates_match:
                n = len(returns_slice)

                effective_returns = np.array([r[1] for r in returns_slice])
                effective_positions = np.array([p[1] for p in positions_slice])
                weights = np.array([self.gamma ** (n - 1 - i) for i in range(n)])

                reward = np.sum(weights * effective_positions * effective_returns)

                log_returns = [
					np.log(1 + r) if p == 1 else 0.0
					for p, r in zip(effective_positions, effective_returns)
				]
                total_return = np.exp(np.sum(log_returns)) - 1.0

                transition_count = np.sum(np.abs(np.diff(effective_positions)))
                transaction_cost = transition_count * self.transaction_cost

                reward -= transaction_cost
                total_return -= transaction_cost
            else:
                print("[Warning] Mismatched dates in return and position histories!")


        self.index_position += 1
        if self.index_position >= len(self.data):
            self.done = True

        obs = self._get_observation()

        info = {
            "date": date_today,
            "position": self.position,
            "mapped_action": mapped_action,
            "action_str": self.action_label[action],
            "return": daily_return,
            "total_return": np.exp(self.cumulative_log_return) - 1,
            "reward": reward
        }

        return obs, reward, self.done, info

    def _get_observation(self):
        indices = self.data.index[self.index_position - self.window_size : self.index_position]
        window = self.data.loc[indices]
        return window.values.flatten().astype(np.float32)

    def close(self):
        pass

In [9]:
# splits = {
#         'train': (0, int(0.75 * len(data))),
#         'test': (int(0.75 * len(data)), int(0.9 * len(data))),
#         'oos': (int(0.9 * len(data)), len(data))
#         }

# train_start, train_end = splits['train']
# scaler = StandardScaler()
# scaler.fit(data.iloc[train_start:train_end])

# scaled_values = scaler.transform(data)
# temp = pd.DataFrame(scaled_values, columns=data.columns, index=data.index)

# Environment Class Debug

In [10]:
env = CSI300TradingEnv(data = data, mode = 'train', lookback = 2, ignored = 0, transaction_cost = transaction_cost)
obs = env.reset()

rows = []

for step in range(30):
    action = random.randint(0, 2)

    obs, reward, done, info = env.step(action)

    row = {
        "Action": f"{action:<2} {info['action_str']:<9}",
        "Mapped": info["mapped_action"],
        "Position": info["position"],
        "Return": round(info["return"], 5),
        "Cumulative_Return": round(info["total_return"], 5),
        "Reward": round(info["reward"], 5)
    }
    rows.append((info["date"], row))

df = pd.DataFrame.from_dict(dict(rows), orient="index")

df.head(30)

,Action,Mapped,Position,Return,Cumulative_Return,Reward
2008-05-21,2 liquidate,1,-1,-0.06211,0.00000,0.00000
2008-05-22,1 enter,0,1,0.01757,0.01605,0.00000
2008-05-23,1 enter,0,1,-0.00931,0.00659,-0.02813
2008-05-26,2 liquidate,1,-1,-0.01484,0.00508,-0.02322
2008-05-27,0 hold,-1,-1,-0.02554,0.00508,0.00918
2008-05-28,0 hold,-1,-1,0.00938,0.00508,0.01360
2008-05-29,2 liquidate,1,-1,0.02314,0.00508,-0.03158
2008-05-30,0 hold,-1,-1,-0.02198,0.00508,0.00116
2008-06-02,2 liquidate,1,-1,0.00408,0.00508,0.01570
2008-06-03,1 enter,0,1,0.00676,0.01036,-0.01043


# Shallow NN Trader

In [11]:
class Shallow_NN_Agent:
    def __init__(self, input_dim, hidden_dims = [128, 64, 32], action_dim = 3, lr = 1e-3, gamma = 0.9, lookback = 50, ignored = 5):
        self.gamma = gamma
        self.lookback = lookback
        self.ignored = ignored
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        layers = []
        prev_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim

        layers.append(nn.Linear(prev_dim, action_dim))
        layers.append(nn.Softmax(dim = -1))
        self.net = nn.Sequential(*layers).to(self.device)
        self.optimizer = optim.Adam(self.net.parameters(), lr = lr)

        self.history = []

    def act(self, filtration, do_debug):
        x = torch.tensor(filtration, dtype=torch.float32, device=self.device)
        probs = self.net(x)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()

        if do_debug:
            print(f"Probs: {probs.detach().cpu().numpy()}, Action: {action.item()}")


        self.history.append((x, action, dist.log_prob(action)))

        return action.item()

    def update(self, reward):
        if len(self.history) < self.lookback:
            return

        useful = self.history[:self.lookback - self.ignored]

        loss = 0
        n = len(useful)

        for t, (x, action, log_prob) in enumerate(useful):
            discounted = reward * (self.gamma ** (n - 1 - t))
            loss -= log_prob * discounted

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.history.clear()

# Training
## Model and Training Variables

In [12]:
lookback = 20
ignored = 3
window_size = 1
gamma = 0.9

transaction_cost = 0.0015

## Environment and Agent Startup

In [13]:
env = CSI300TradingEnv(
    data=data,
    window_size=window_size,
    lookback=lookback,
    ignored=ignored,
    transaction_cost=transaction_cost,
    gamma=gamma,
    mode="train"
)

agent = Shallow_NN_Agent(
    input_dim=env.observation_space.shape[0],
    hidden_dims=[128, 64, 32],
    action_dim=3,
    lr=1e-3,
    gamma=gamma,
    lookback=lookback,
    ignored=ignored
)

## Training

In [18]:
env.set_mode("train")
obs = env.reset()
done = False

rows = []

while not done:
    action = agent.act(obs, False)
    obs, reward, done, info = env.step(action)

    if rows:
        rows[-1][1]["Return"] = round(info["return"], 5)

    row = {
        "Action": f"{action:<2} {info['action_str']:<9}",
        "Mapped": info["mapped_action"],
        "Position": info["position"],
        "Return": None,
        "Cumulative_Return": round(info["total_return"], 5),
        "Reward": round(info["reward"], 5)
    }
    rows.append((info["date"], row))

    if reward != 0:
        agent.update(reward)


df_train = pd.DataFrame.from_dict(dict(rows), orient="index")

df_train.index = df_train.index.strftime("%Y-%m-%d")
df_train.tail()

,Action,Mapped,Position,Return,Cumulative_Return,Reward
2020-09-29,2 liquidate,1,-1,-0.00016,0.57045,0.01487
2020-09-30,2 liquidate,1,-1,0.01216,0.57045,0.02429
2020-10-09,0 hold,-1,-1,0.00892,0.57045,0.02122
2020-10-12,0 hold,-1,-1,0.02289,0.57045,0.00991
2020-10-13,0 hold,-1,-1,NaN,0.57045,0.00986


In [19]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=100 * df_train["Cumulative_Return"],
    mode='lines+markers',
    name='Cumulative Return'
))

fig.update_layout(
    title="Agent Cumulative Return (Training Set)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    yaxis=dict(
        tickformat=".2f",
        ticksuffix="%"
    ),
    xaxis=dict(
        type="category",
        tickmode='linear',
        tick0=0,
        dtick=max(len(df_train) // 15, 1),
        showgrid=False
    ),
    template="plotly_dark",
    height=500
)

fig.show()

# Test

In [20]:
env.set_mode("test")
obs = env.reset()
done = False

rows = []

while not done:
    action = agent.act(obs, False)  # Use the trained agent
    obs, reward, done, info = env.step(action)

    # Apply today's return to yesterday's row
    if rows:
        rows[-1][1]["Return"] = round(info["return"], 5)

    row = {
        "Action": f"{action:<2} {info['action_str']:<9}",
        "Mapped": info["mapped_action"],
        "Position": info["position"],
        "Return": None,  # Will be filled next step
        "Cumulative_Return": round(info["total_return"], 5),
        "Reward": round(info["reward"], 5)
    }
    rows.append((info["date"], row))

# Final test log as DataFrame
df_test = pd.DataFrame.from_dict(dict(rows), orient="index")

In [21]:
df_test.index = pd.to_datetime(df_test.index).strftime("%Y-%m-%d")

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=100 * df_test["Cumulative_Return"],
    mode='lines+markers',
    name='Cumulative Return'
))

fig.update_layout(
    title="Agent Cumulative Return Over Time (Test Set)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    yaxis=dict(
        tickformat=".2f",
        ticksuffix="%"
    ),
    xaxis=dict(
        type="category",
        tickmode='linear',
        tick0=0,
        dtick=max(len(df_test) // 15, 1),
        tickangle=-45,
        showgrid=False
    ),
    template="plotly_dark",
    height=500
)

fig.show()

In [ ]:
print(df_test["Action"].value_counts())

Action
1  enter        306
0  hold         250
2  liquidate     41
Name: count, dtype: int64
